In [1]:
'''
=====Experiment=====
Dataset: UNSW-NB15 dataset

Training in chronological window subgraph
Downsample 90% normal traffic stratigically
Split train and test subgraph stratigically 

IP as node
Encode Ports in raw numbers
'''

from torch_geometric.utils import from_networkx, add_self_loops, degree
from torch_geometric.nn import MessagePassing
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.loader import NeighborSampler
import torch.nn as nn
import torch as th
import torch.nn.functional as F
# import dgl.function as fn
import networkx as nx
import pandas as pd
import socket
import struct
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
# import seaborn as sns
# import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from torch_geometric.loader import DataLoader


project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

from Datasets.UNSW_NB15.UNSW_NB15_config import UNSW_NB15_Config

In [ ]:
csv_file_name = "all_raw"

data = pd.read_csv(os.path.join(project_root, "Datasets", f"UNSW_NB15/All/{csv_file_name}.csv"))

DATASET_NAME = "UNSW_NB15"
EXPERIMENT_NAME = "window_raw_port"

SOURCE_FILE_ID_COL_NAME = UNSW_NB15_Config.SOURCE_FILE_ID_COL_NAME

SOURCE_IP_COL_NAME = UNSW_NB15_Config.SOURCE_IP_COL_NAME
DESTINATION_IP_COL_NAME = UNSW_NB15_Config.DESTINATION_IP_COL_NAME
SOURCE_PORT_COL_NAME = UNSW_NB15_Config.SOURCE_PORT_COL_NAME
DESTINATION_PORT_COL_NAME = UNSW_NB15_Config.DESTINATION_PORT_COL_NAME

ATTACK_CLASS_COL_NAME = UNSW_NB15_Config.ATTACK_CLASS_COL_NAME
IS_ATTACK_COL_NAME = UNSW_NB15_Config.IS_ATTACK_COL_NAME

BENIGN_CLASS_NAME = UNSW_NB15_Config.BENIGN_CLASS_NAME

TIME_COLS = UNSW_NB15_Config.TIME_COL_NAMES

CATEGORICAL_COLS = UNSW_NB15_Config.CATEGORICAL_COLS

print(data[ATTACK_CLASS_COL_NAME].value_counts())
print(data[IS_ATTACK_COL_NAME].value_counts())

MULTICLASS = True

if MULTICLASS:
    label_col = ATTACK_CLASS_COL_NAME
    data.drop(columns=[IS_ATTACK_COL_NAME], inplace=True)
else:
    label_col = IS_ATTACK_COL_NAME
    data.drop(columns=[ATTACK_CLASS_COL_NAME], inplace=True)


saves_path = os.path.join(project_root, "Models/E_GraphSAGE/logs", DATASET_NAME, EXPERIMENT_NAME)

checkpoint_path = os.path.join(saves_path, f"checkpoints_{csv_file_name}.pth")
best_model_path = os.path.join(saves_path, f"best_model_{csv_file_name}.pth")

os.makedirs(saves_path, exist_ok=True)

attack_cat
Normal            2218764
Generic            215481
Exploits            44525
Fuzzers             24246
DoS                 16353
Reconnaissance      13987
Analysis             2677
Backdoors            2329
Shellcode            1511
Worms                 174
Name: count, dtype: int64
label
0    2218764
1     321283
Name: count, dtype: int64


In [3]:
data.drop(columns=UNSW_NB15_Config.DROP_COLS,inplace=True)
print(data.columns)

Index(['srcip', 'sport', 'dstip', 'dsport', 'state', 'dur', 'sbytes', 'dbytes',
       'sttl', 'dttl', 'sloss', 'dloss', 'Sload', 'Dload', 'Spkts', 'Dpkts',
       'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'source_file_id'],
      dtype='object')


In [ ]:
data = pd.get_dummies(data, columns = CATEGORICAL_COLS) # One Hot Encoding for categorical data

converted_categorical_cols = [col for col in data.columns if col.startswith(tuple(CATEGORICAL_COLS))]

In [5]:
print(data.head)

<bound method NDFrame.head of                      srcip  sport              dstip  dsport        dur  \
0            10.40.182.1_0      0        224.0.0.5_0       0  50.004337   
1             10.40.85.1_0      0        224.0.0.5_0       0  50.004341   
2            10.40.182.1_0      0        224.0.0.5_0       0  50.004337   
3             10.40.85.1_0      0        224.0.0.5_0       0  50.004341   
4        192.168.241.243_0    259  192.168.241.243_0   49320   0.000000   
...                    ...    ...                ...     ...        ...   
2540042       59.166.0.0_3   2111    149.171.126.5_3      53   0.001035   
2540043       59.166.0.5_3  49044    149.171.126.3_3   30639   0.220630   
2540044       59.166.0.6_3  37717    149.171.126.7_3   35667   0.031576   
2540045       59.166.0.2_3   1768    149.171.126.7_3   64122   0.096835   
2540046       59.166.0.9_3   7045    149.171.126.7_3      25   0.201886   

         sbytes  dbytes  sttl  dttl  sloss  ...  state_INT  state_MAS

In [6]:
data = data.reset_index()
data.replace([np.inf, -np.inf], np.nan,inplace = True)
data.fillna(0,inplace = True)
data.drop(columns=['index'],inplace=True)
print(data.head)

<bound method NDFrame.head of                      srcip  sport              dstip  dsport        dur  \
0            10.40.182.1_0      0        224.0.0.5_0       0  50.004337   
1             10.40.85.1_0      0        224.0.0.5_0       0  50.004341   
2            10.40.182.1_0      0        224.0.0.5_0       0  50.004337   
3             10.40.85.1_0      0        224.0.0.5_0       0  50.004341   
4        192.168.241.243_0    259  192.168.241.243_0   49320   0.000000   
...                    ...    ...                ...     ...        ...   
2540042       59.166.0.0_3   2111    149.171.126.5_3      53   0.001035   
2540043       59.166.0.5_3  49044    149.171.126.3_3   30639   0.220630   
2540044       59.166.0.6_3  37717    149.171.126.7_3   35667   0.031576   
2540045       59.166.0.2_3   1768    149.171.126.7_3   64122   0.096835   
2540046       59.166.0.9_3   7045    149.171.126.7_3      25   0.201886   

         sbytes  dbytes  sttl  dttl  sloss  ...  state_INT  state_MAS

In [7]:
scaler = StandardScaler()
cols_to_norm = UNSW_NB15_Config.COLS_TO_NORM + [SOURCE_PORT_COL_NAME, DESTINATION_PORT_COL_NAME]
print(data[cols_to_norm].describe()) # Check if there's any too large value

                dur        sbytes        dbytes          sttl          dttl  \
count  2.540047e+06  2.540047e+06  2.540047e+06  2.540047e+06  2.540047e+06   
mean   6.587916e-01  4.339600e+03  3.642759e+04  6.278197e+01  3.076681e+01   
std    1.392493e+01  5.640599e+04  1.610960e+05  7.462277e+01  4.285089e+01   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    1.037000e-03  2.000000e+02  1.780000e+02  3.100000e+01  2.900000e+01   
50%    1.586100e-02  1.470000e+03  1.820000e+03  3.100000e+01  2.900000e+01   
75%    2.145545e-01  3.182000e+03  1.489400e+04  3.100000e+01  2.900000e+01   
max    8.786638e+03  1.435577e+07  1.465753e+07  2.550000e+02  2.540000e+02   

              sloss         dloss         Sload         Dload         Spkts  \
count  2.540047e+06  2.540047e+06  2.540047e+06  2.540047e+06  2.540047e+06   
mean   5.163921e+00  1.632944e+01  3.695645e+07  2.450861e+06  3.328884e+01   
std    2.251707e+01  5.659474e+01  1.186043e+08  4.

In [8]:
def check_numeric_issues(df, cols_to_norm):
    for col in cols_to_norm:
        try:
            # Try to coerce to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Try to clip the column
            df[col] = df[col].clip(lower=-1e9, upper=1e9)
            
        except Exception as e:
            print(f"❌ Column '{col}' failed with error: {e}")
            print(f"  - Sample values: {df[col].dropna().unique()[:5]}")
            print(f"  - Data type: {df[col].dtype}")
            continue

    print("\n✅ All other columns processed successfully.")

check_numeric_issues(data, UNSW_NB15_Config.COLS_TO_NORM)


✅ All other columns processed successfully.


In [9]:
data[cols_to_norm] = scaler.fit_transform(data[cols_to_norm])

In [10]:
from sklearn.preprocessing import LabelEncoder

num_classes = 2
class_map = [0, 1]
if MULTICLASS:
    le = LabelEncoder()
    attack_labels = le.fit_transform(data[ATTACK_CLASS_COL_NAME])
    class_map = le.classes_
    print(class_map)
    print("Attack label mapping:", dict(zip(class_map, range(len(class_map)))))
    data[ATTACK_CLASS_COL_NAME] = attack_labels
    num_classes = len(class_map)
    class_dict = {le.inverse_transform([i])[0]: i for i in range(len(le.classes_))}

BENIGN_CLASS_LABEL = le.transform([BENIGN_CLASS_NAME])[0] if MULTICLASS else 0
ADVERSARIAL_CLASS_LABEL = len(class_map)

['Analysis' 'Backdoors' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
Attack label mapping: {'Analysis': 0, 'Backdoors': 1, 'DoS': 2, 'Exploits': 3, 'Fuzzers': 4, 'Generic': 5, 'Normal': 6, 'Reconnaissance': 7, 'Shellcode': 8, 'Worms': 9}


In [11]:
# # Maintain the order of the rows in the original dataframe

feature_cols = cols_to_norm + converted_categorical_cols

print('Feature Columns:', feature_cols)
num_features = len(feature_cols)
print('Number of Features:', num_features)

data['h'] = data[ feature_cols ].values.tolist()

Feature Columns: ['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'sport', 'dsport', 'state_ACC', 'state_CLO', 'state_CON', 'state_ECO', 'state_ECR', 'state_FIN', 'state_INT', 'state_MAS', 'state_PAR', 'state_REQ', 'state_RST', 'state_TST', 'state_TXD', 'state_URH', 'state_URN', 'state_no']
Number of Features: 56


In [12]:
def create_graph(df):

    G_nx = nx.from_pandas_edgelist(df, SOURCE_IP_COL_NAME, DESTINATION_IP_COL_NAME, ['h', label_col], create_using=nx.MultiDiGraph())
    G_pyg = from_networkx(G_nx)

    num_nodes = G_pyg.num_nodes
    num_edges = G_pyg.num_edges

    G_pyg.x = th.ones(num_nodes, len(df['h'].iloc[0])) 

    edge_attr_list = []
    edge_label_list = []

    for u, v, key, data in G_nx.edges(keys=True, data=True):
        edge_attr_list.append(data['h']) 
        edge_label_list.append(data[label_col]) 

    G_pyg.edge_attr = th.tensor(edge_attr_list, dtype=th.float32)
    G_pyg.edge_label = th.tensor(edge_label_list, dtype=th.long)

    return G_pyg

In [13]:
from collections import defaultdict
import heapq

import tqdm

class Downsampler:
    def __init__(self, downsample_classes=[BENIGN_CLASS_LABEL], downsample_ratios=[0.1]):
        """
        downsample_classes: list of class names to downsample
        downsample_ratio: keep no more than this ratio for each class
        """
        assert len(downsample_classes) == len(downsample_ratios)
        self.downsample_classes = downsample_classes
        self.downsample_ratio = downsample_ratios

    def downsample(self, label_counts_list, X, y):
        total_counts = defaultdict(int)

        class_heaps = defaultdict(list)
        for i, lc in enumerate(label_counts_list):
            for cls in self.downsample_classes:
                class_label_count = lc.get(cls, 0)
                total_counts[cls] += class_label_count
                heapq.heappush(class_heaps[cls], (-class_label_count, i))

        class_target = {
            cls: total_counts[cls] * self.downsample_ratio[i] for i, cls in enumerate(self.downsample_classes)
        }
                
        indices_to_remove = set()
        class_counts = total_counts

        # 3. For each class, remove top contributing samples until threshold reached
        for cls in self.downsample_classes:
            target = class_target[cls]
            heap = class_heaps[cls]

            pbar = tqdm(desc=f"Downsampling '{cls}'", total=len(heap))
            while class_counts[cls] > target and heap:
                _, idx = heapq.heappop(heap)
                if idx in indices_to_remove:
                    continue
                # For each class in this sample, if it's a downsample class, decrement the count
                for sample_cls, count in label_counts_list[idx].items():
                    if sample_cls in self.downsample_classes:
                        class_counts[sample_cls] -= count
                indices_to_remove.add(idx)
                pbar.update(1)
                pbar.set_postfix(class_label=cls, remaining=class_counts[cls], target=target)
            pbar.close()

        # 4. Apply filter
        keep_mask = [i for i in range(len(X)) if i not in indices_to_remove]
        X_new = [X[i] for i in keep_mask]
        y_new = [y[i] for i in keep_mask]

        return X_new, y_new

downsampler = Downsampler()

In [14]:
from collections import defaultdict
from typing import Counter
from sklearn.preprocessing import MultiLabelBinarizer

from tqdm import tqdm

class StratifiedGraphDataset:

    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.total_count = len(self.y)

        # Compute class weights
        labels = []

        for graph in self.X:
            labels.append(graph.edge_label.tolist())

        labels = np.concatenate(labels)

        self.class_counts = Counter(labels)

        # Compute the class weights
        self.class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(labels),
            y=labels
        )

    def k_fold_split(self, k: int = 5, test_ratio: float = 0.15, random_state: int = 42):
        cv = MultilabelStratifiedShuffleSplit(test_size=test_ratio, random_state=random_state, n_splits=k)

        mlb = MultiLabelBinarizer()

        y_binary = mlb.fit_transform(self.y)

        return cv.split(np.zeros(len(self.X)), y_binary)

    def graph_train_test_split(self, test_ratio: float = 0.15, random_state: int = 42):
        train_idx, test_idx = next(self.k_fold_split(k = 1, test_ratio = test_ratio, random_state = random_state))
        
        X_train = [self.X[i] for i in train_idx]
        X_test = [self.X[i] for i in test_idx]

        y_train = [self.y[i] for i in train_idx]
        y_test = [self.y[i] for i in test_idx]

        return StratifiedGraphDataset(X_train, y_train), StratifiedGraphDataset(X_test, y_test)
    
    def print_class_distribution_and_weights(self):
        # Use the label encoder to inverse transform the class labels
        class_counts_named = {cls: count for cls, count in self.class_counts.items()}
        class_weights_named = {cls: weight for cls, weight in enumerate(self.class_weights)}
        print("Class Counts and Weights:")
        for cls_label in class_counts_named.keys():
            count = class_counts_named[cls_label]
            weight = class_weights_named[cls_label]
            print(f"{cls_label:<2}  {le.inverse_transform([cls_label])[0]:<15}: Count = {count:<10}, Weight = {weight:<10.4f}")
    
    def __len__(self):
        return self.total_count

    def __iter__(self):
        for g in self.X:
            yield g
            
    def __getitem__(self, idx):
        if isinstance(idx, int):
            return self.X[idx], self.y[idx]
        elif isinstance(idx, slice):
            return [self.X[i] for i in range(len(self.X))][idx], [self.y[i] for i in range(len(self.y))][idx]
        else:
            raise TypeError("Index must be an integer or a slice.")

def generate_graph_datasets(
    df: pd.DataFrame, 
    window_size: int = 200, 
    # overlap_ratio: float = 0, 
    feature_cols=feature_cols,
    ordering_cols=[SOURCE_FILE_ID_COL_NAME] + TIME_COLS + [ATTACK_CLASS_COL_NAME], 
    label_col=label_col,
    build_graph_func=create_graph,
    downsampler=downsampler
    ):

    print("All Columns: ", df.columns)
    print("Ordering Columns: ", ordering_cols)
    assert all(col in df.columns for col in ordering_cols), "All timestamp columns are required"
    assert label_col in df.columns, "Edge label column 'label' is required"
    
    df = df.sort_values(ordering_cols).reset_index(drop=True)
    window_size = int(window_size)
    
    df.drop(columns=set(df.columns) - set(feature_cols) - set(label_col))

    print("Final Columns: ", df.columns)
    
    label_counts_list = []
    X = []
    y = []

    progress_bar = tqdm(range(0, len(df), window_size), desc=f"Generating graphs")
    for start in progress_bar:
        window_df = df[start: min(start + window_size, len(df))]
        contains_label = window_df[label_col].unique()

        G_pyg = build_graph_func(window_df)

        label_counts = window_df[label_col].value_counts()

        label_counts_list.append(label_counts)
        X.append(G_pyg)
        y.append(contains_label.tolist())

    X, y = downsampler.downsample(label_counts_list, X, y)
        
    return StratifiedGraphDataset(X, y)

In [15]:
graph_dataset = generate_graph_datasets(data)
full_train_graph_dataset, test_graph_dataset = graph_dataset.graph_train_test_split(test_ratio=0.15, random_state=42)


All Columns:  Index(['srcip', 'sport', 'dstip', 'dsport', 'dur', 'sbytes', 'dbytes', 'sttl',
       'dttl', 'sloss', 'dloss', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin',
       'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'source_file_id',
       'state_ACC', 'state_CLO', 'state_CON', 'state_ECO', 'state_ECR',
       'state_FIN', 'state_INT', 'state_MAS', 'state_PAR', 'state_REQ',
       'state_RST', 'state_TST', 'state_TXD', 'state_URH', 'state_URN',
       'state_no', 'h'],
      dtype='object')
Ordering Columns:  ['source_file_id', 'Stime', 'Ltime', 'attack_cat']
Final Columns:  Index(['srcip', 'sport', 'dstip', 'dsport', 

Downsampling '6':  85%|████████▌ | 10853/12701 [00:10<00:01, 1058.07it/s, class_label=6, remaining=221852, target=2.22e+5]


In [16]:
print("Class Distrubution:", data[label_col].value_counts())

print("Number of graphs after downsampling:", len(graph_dataset))
graph_dataset.print_class_distribution_and_weights()
print("Number of training graphs:", len(full_train_graph_dataset))
full_train_graph_dataset.print_class_distribution_and_weights()
print("Number of testing graphs:", len(test_graph_dataset))
test_graph_dataset.print_class_distribution_and_weights()

Class Distrubution: attack_cat
6    2218764
5     215481
3      44525
4      24246
2      16353
7      13987
0       2677
1       2329
8       1511
9        174
Name: count, dtype: int64
Number of graphs after downsampling: 1848
Class Counts and Weights:
6   Normal         : Count = 221852    , Weight = 0.1665    
5   Generic        : Count = 105186    , Weight = 0.3512    
3   Exploits       : Count = 17858     , Weight = 2.0688    
0   Analysis       : Count = 2083      , Weight = 17.7363   
1   Backdoors      : Count = 1818      , Weight = 20.3216   
2   DoS            : Count = 11800     , Weight = 3.1309    
4   Fuzzers        : Count = 5177      , Weight = 7.1363    
7   Reconnaissance : Count = 3401      , Weight = 10.8629   
9   Worms          : Count = 41        , Weight = 901.0902  
8   Shellcode      : Count = 231       , Weight = 159.9338  
Number of training graphs: 1570
Class Counts and Weights:
6   Normal         : Count = 188281    , Weight = 0.1667    
5   Generic     

In [17]:
class SAGELayerPyG(MessagePassing):
    def __init__(self, in_channels, edge_dim, out_channels, activation=F.relu):
        super().__init__(aggr='mean')  # mean aggregation
        self.W_msg = nn.Linear(in_channels + edge_dim, out_channels)
        self.W_apply = nn.Linear(in_channels + out_channels, out_channels)
        self.activation = activation

    def forward(self, x, edge_index, edge_attr):
        # x: [num_nodes, in_channels]
        # edge_attr: [num_edges, edge_dim]
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        # x_j: features of source nodes (neighbours)
        msg_input = th.cat([x_j, edge_attr], dim=1)
        return self.W_msg(msg_input)

    def update(self, aggr_out, x):
        # aggr_out: [num_nodes, out_channels]
        combined = th.cat([x, aggr_out], dim=1)
        out = self.W_apply(combined)
        return self.activation(out)
    
class MLPPredictor(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MLPPredictor, self).__init__()
        self.lin = nn.Linear(in_channels * 2, out_channels)

    def forward(self, data, z):
        row, col = data.edge_index
        # Concatenate the features of source and target nodes for each edge
        edge_feat = th.cat([z[row], z[col]], dim=1)
        return self.lin(edge_feat)

class EGraphSAGE(nn.Module):
    def __init__(self, node_in_channels, edge_in_channels, hidden_channels, out_channels, dropout=0.2):
        super(EGraphSAGE, self).__init__()
        self.dropout = dropout
        self.conv1 = SAGELayerPyG(node_in_channels, edge_in_channels, hidden_channels)
        self.conv2 = SAGELayerPyG(hidden_channels, edge_in_channels, hidden_channels)
        self.mlp_predictor = MLPPredictor(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=self.dropout)
        x = self.conv2(x, edge_index, edge_attr)
        return self.mlp_predictor(data, x)

In [18]:
device = th.device("cuda:0" if th.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
th.cuda.empty_cache()

In [20]:


def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)



In [21]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

def grid_search(graph_dataset, epochs, learning_rates, hidden_dims, folds=3):
    global num_features
    
    best_params = {}
    best_f1 = 0

    # Precompute the train and validation graphs for all folds
    folds_list = []
    for i in range(folds):
        train_graph_dataset, val_graph_dataset = graph_dataset.graph_train_test_split(test_ratio=0.15, random_state=i)
        folds_list.append((train_graph_dataset, val_graph_dataset))

    params_results = {}
    for lr in learning_rates:
        for hidden_dim in hidden_dims:
            for drop_out in drop_outs:
                print(f"Testing with learning rate: {lr}, hidden_dim: {hidden_dim}")
                fold_f1_scores = []

                for fold, (train_graph_dataset, val_graph_dataset) in enumerate(folds_list):
                    print(f"Fold {fold + 1}")

                    model = EGraphSAGE(node_in_channels=num_features,
                                    edge_in_channels=num_features,
                                    hidden_channels=hidden_dim,
                                    dropout = drop_out,
                                    out_channels=num_classes).to(device)

                    model.apply(init_weights)

                    # Normalize to stabilize training
                    class_weights = th.FloatTensor(train_graph_dataset.class_weights).to(device)
                    print("Class weights:", class_weights)

                    criterion = nn.CrossEntropyLoss(weight=class_weights)
                    optimizer = th.optim.Adam(model.parameters(), lr=lr)
                    scheduler = th.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

                    best_epoch_f1 = 0  # Track the best F1 score for this fold

                    for epoch in range(epochs):
                        try:
                            total_train_loss = 0
                            total_val_loss = 0

                            model.train()
                            for G_pyg_train in tqdm(train_graph_dataset, desc="Training", leave=False):
                                G_pyg_train = G_pyg_train.to(device)

                                G_pyg_train.edge_label = G_pyg_train.edge_label.to(device)
                                G_pyg_train.edge_attr = G_pyg_train.edge_attr.to(device)
                                
                                out = model(G_pyg_train)
                                loss = criterion(out, G_pyg_train.edge_label)
                                total_train_loss += loss.item()

                                optimizer.zero_grad()
                                loss.backward()

                                optimizer.step()
                                scheduler.step()
                            
                            model.eval()

                            test_label_list = []
                            pred_label_list = []

                            with th.no_grad():
                                for G_pyg_val in tqdm(val_graph_dataset, desc="Validation", leave=False):

                                    G_pyg_val.to(device)
                                    G_pyg_val.edge_label = G_pyg_val.edge_label.to(device)
                                    G_pyg_val.edge_attr = G_pyg_val.edge_attr.to(device)

                                    out = model(G_pyg_val)
                                    loss = criterion(out, G_pyg_val.edge_label)
                                    total_val_loss += loss.item()

                            test_label = th.cat(test_label_list)
                            pred_label = th.cat(pred_label_list)

                            val_f1 = f1_score(test_label.cpu(), pred_label.cpu(), average='weighted')

                            if val_f1 > best_epoch_f1:
                                best_epoch_f1 = val_f1  # Update the best F1 score for this fold
                                print(f"Best F1 Score at epoch {epoch}: {best_epoch_f1:.4f}, Train Loss: {total_train_loss:.4f}, Validation Loss: {total_val_loss:.4f}, Parameters: lr={lr}, hidden_dim={hidden_dim}, drop_out={drop_out}")

                            print(f'Epoch {epoch}, Train Loss: {total_train_loss:.4f}, Validation Loss: {total_val_loss:.4f}, Validation F1: {avg_f1:.4f}')

                        except Exception as e:
                            print(f"An error occurred at epoch {epoch}: {str(e)}")
                            break

                    fold_f1_scores.append(best_epoch_f1)  # Append the best F1 score for this fold
                
                avg_f1 = sum(fold_f1_scores) / len(fold_f1_scores)
                params_results[(drop_out, lr, hidden_dim)] = {'folds': fold_f1_scores, 'avg_f1': avg_f1}
                print(f"Average F1 Score for drop_out {drop_out}, learning rate {lr}, hidden_dim {hidden_dim}: {avg_f1:.4f}")

                if avg_f1 > best_f1:
                    best_f1 = avg_f1
                    best_params = {'learning_rate': lr, 'hidden_dim': hidden_dim, 'drop_out': drop_out}

    print(f"Best Parameters: {best_params}, Best F1 Score: {best_f1:.4f}")
    print("All results:", params_results)


learning_rates = [0.001, 0.005, 0.01, 0.05]
hidden_dims = [128, 256, 512]
drop_outs = [0.2, 0.3, 0.4]

# grid_search(full_train_graph_dataset, epochs=10, learning_rates=learning_rates, hidden_dims=hidden_dims, drop_outs=drop_outs)


In [22]:
train_graph_dataset, val_graph_dataset = full_train_graph_dataset.graph_train_test_split(test_ratio=0.15, random_state=42)

In [23]:
import pickle

def save_progress(epoch, model, optimizer, best_f1, train_loss_history, val_loss_history, val_f1_history, saved_model_epochs):
    global checkpoint_path, train_loss_history_path, val_loss_history_path, val_f1_history_path, saved_model_epochs_path

    # Save checkpoint
    th.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_f1': best_f1
    }, checkpoint_path)
    with open(train_loss_history_path, 'wb') as f:
        pickle.dump(train_loss_history, f)
    with open(val_loss_history_path, 'wb') as f:
        pickle.dump(val_loss_history, f)
    with open(val_f1_history_path, 'wb') as f:
        pickle.dump(val_f1_history, f)
    with open(saved_model_epochs_path, 'wb') as f:
        pickle.dump(saved_model_epochs, f)

In [24]:
# Extract the best parameters from the grid search
import pickle


best_hidden_dim = 256  # Replace with the best hidden_dim found
best_learning_rate = 0.01  # Replace with the best learning_rate found
best_drop_out = 0.3  # Replace with the best drop_out found
epochs = 1000

print("Number of train graphs: ", len(train_graph_dataset))

# Initialize the model with the best parameters
model = EGraphSAGE(node_in_channels=num_features, 
                   edge_in_channels=num_features,
                   hidden_channels=best_hidden_dim,
                   dropout = best_drop_out,
                   out_channels=num_classes).to(device)

model.apply(init_weights)

# Normalize class weights
class_weights = th.FloatTensor(train_graph_dataset.class_weights).to(device)
print("Class weights:", class_weights)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = th.optim.Adam(model.parameters(), lr=best_learning_rate)
scheduler = th.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

# ===== Load checkpoint if exists =====
best_f1 = 0
start_epoch = 0

if os.path.exists(checkpoint_path):
    checkpoint = th.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_f1 = checkpoint['best_f1']
    print(f"Resumed training from epoch {start_epoch}")

train_loss_history = []
val_loss_history = []
val_f1_history = []
saved_model_epochs = []

train_loss_history_path = os.path.join(saves_path, 'train_loss_history.pkl')
val_loss_history_path = os.path.join(saves_path, 'val_loss_history.pkl')
val_f1_history_path = os.path.join(saves_path, 'val_f1_history.pkl')
saved_model_epochs_path = os.path.join(saves_path, 'saved_model_epochs.pkl')

if os.path.exists(train_loss_history_path) and os.path.exists(val_loss_history_path) and os.path.exists(val_f1_history_path) and os.path.exists(saved_model_epochs_path):
    with open(train_loss_history_path, 'rb') as f:
        train_loss_history = pickle.load(f)
    with open(val_loss_history_path, 'rb') as f:
        val_loss_history = pickle.load(f)
    with open(val_f1_history_path, 'rb') as f:
        val_f1_history = pickle.load(f)
    with open(saved_model_epochs_path, 'rb') as f:
        saved_model_epochs = pickle.load(f)

# ===== Start Training =====
for epoch in range(start_epoch, epochs):
    model.train()

    train_loss = 0
    num_train_graphs = len(train_graph_dataset)

    val_loss = 0
    num_val_graphs = len(val_graph_dataset)

    optimizer.zero_grad()
    for G_pyg_train in tqdm(train_graph_dataset, desc="Training", leave=False):

        # Move the graph data to the device
        G_pyg_train = G_pyg_train.to(device)
        G_pyg_train.edge_label = G_pyg_train.edge_label.to(device)
        G_pyg_train.edge_attr = G_pyg_train.edge_attr.to(device)

        out = model(G_pyg_train)
        loss = criterion(out, G_pyg_train.edge_label) / num_train_graphs
        
        train_loss += loss.item()
        loss.backward()

    optimizer.step()
    scheduler.step()

    test_label_list = []
    pred_label_list = []

    model.eval()
    with th.no_grad():
        for G_pyg_val in tqdm(val_graph_dataset, desc="Evaluation", leave=False):
            G_pyg_val = G_pyg_val.to(device)
            G_pyg_val.edge_label = G_pyg_val.edge_label.to(device)
            G_pyg_val.edge_attr = G_pyg_val.edge_attr.to(device)

            out = model(G_pyg_val)
            loss = criterion(out, G_pyg_val.edge_label) / num_val_graphs
            val_loss += loss.item()

            test_label_list.append(G_pyg_val.edge_label.cpu())
            pred_label_list.append(out.argmax(dim=1).cpu())

    test_label = th.cat(test_label_list)
    pred_label = th.cat(pred_label_list)

    val_f1 = f1_score(test_label, pred_label, average='weighted')
    val_f1_micro = f1_score(test_label, pred_label, average='micro')
    val_f1_macro = f1_score(test_label, pred_label, average='macro')
    
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)
    val_f1_history.append((val_f1, val_f1_micro, val_f1_macro))

    if val_f1 > best_f1:
        best_f1 = val_f1  # Update the best F1 score for this fold
        best_model_state = model.state_dict()
        saved_model_epochs.append(epoch)

        save_progress(epoch, model, optimizer, best_f1, train_loss_history, val_loss_history, val_f1_history, saved_model_epochs)
        th.save(best_model_state, best_model_path)
        print(f"Epoch {epoch} Saved best model. Best F1:", best_f1)

    if epoch % 10 == 0:
        # Save checkpoint
        print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation F1: {val_f1:.4f}')
        save_progress(epoch, model, optimizer, best_f1, train_loss_history, val_loss_history, val_f1_history, saved_model_epochs)

# Save the trained model
print("Model training completed and saved.")

Number of train graphs:  1334
Class weights: tensor([1.8187e+01, 2.1024e+01, 3.0260e+00, 2.0663e+00, 7.5452e+00, 3.5045e-01,
        1.6674e-01, 1.0824e+01, 1.5976e+02, 8.8933e+02], device='cuda:0')


Epoch 0 Saved best model. Best F1: 0.5515892648773498
Epoch 0, Train Loss: 2.5758, Validation Loss: 9.1593, Validation F1: 0.5516


Epoch 4 Saved best model. Best F1: 0.8118737158032682


Epoch 5 Saved best model. Best F1: 0.853744781159511


Epoch 7 Saved best model. Best F1: 0.8757121971568889


Epoch 9 Saved best model. Best F1: 0.8772209726735137


Epoch 10, Train Loss: 1.1335, Validation Loss: 1.1755, Validation F1: 0.8480


Epoch 11 Saved best model. Best F1: 0.8802975079672832


Epoch 12 Saved best model. Best F1: 0.8943944105661302


Epoch 13 Saved best model. Best F1: 0.9031615558788516


Epoch 20, Train Loss: 0.9268, Validation Loss: 0.8559, Validation F1: 0.8979


Epoch 23 Saved best model. Best F1: 0.9054066565291162


Epoch 27 Saved best model. Best F1: 0.9059662922654745


Epoch 30, Train Loss: 0.8516, Validation Loss: 0.7919, Validation F1: 0.8948


Epoch 33 Saved best model. Best F1: 0.9068422822001174


Epoch 38 Saved best model. Best F1: 0.9081792581777969


Epoch 40, Train Loss: 0.8047, Validation Loss: 0.7689, Validation F1: 0.9051


Epoch 41 Saved best model. Best F1: 0.9083480399305657


Epoch 47 Saved best model. Best F1: 0.9097300748499992


Epoch 50, Train Loss: 0.7810, Validation Loss: 0.7398, Validation F1: 0.9075


Epoch 52 Saved best model. Best F1: 0.9099552199260699


Epoch 60, Train Loss: 0.7502, Validation Loss: 0.7537, Validation F1: 0.9092


Epoch 70, Train Loss: 0.7334, Validation Loss: 0.7374, Validation F1: 0.9095


Epoch 73 Saved best model. Best F1: 0.9113193961819851


Epoch 80, Train Loss: 0.7225, Validation Loss: 0.7207, Validation F1: 0.9076


Epoch 82 Saved best model. Best F1: 0.9121619013192771


Epoch 87 Saved best model. Best F1: 0.9126158099920648


Epoch 90, Train Loss: 0.6952, Validation Loss: 0.7056, Validation F1: 0.9096


Epoch 95 Saved best model. Best F1: 0.9149761929463381


Epoch 96 Saved best model. Best F1: 0.9160522586079959


Epoch 100, Train Loss: 0.6752, Validation Loss: 0.7310, Validation F1: 0.9067


Epoch 110, Train Loss: 0.6590, Validation Loss: 0.6923, Validation F1: 0.9041


Epoch 120, Train Loss: 0.6460, Validation Loss: 0.6892, Validation F1: 0.9063


Epoch 130, Train Loss: 0.6295, Validation Loss: 0.6976, Validation F1: 0.9059


Epoch 140, Train Loss: 0.6235, Validation Loss: 0.6728, Validation F1: 0.9086


Epoch 150, Train Loss: 0.6061, Validation Loss: 0.6940, Validation F1: 0.9121


Epoch 160, Train Loss: 0.5990, Validation Loss: 0.6776, Validation F1: 0.9099


Epoch 170, Train Loss: 0.5960, Validation Loss: 0.6844, Validation F1: 0.9110


Epoch 180, Train Loss: 0.5932, Validation Loss: 0.6790, Validation F1: 0.9109


Epoch 190, Train Loss: 0.5810, Validation Loss: 0.7065, Validation F1: 0.9093


Epoch 200, Train Loss: 0.5678, Validation Loss: 0.6999, Validation F1: 0.9127


Epoch 210, Train Loss: 0.5615, Validation Loss: 0.7001, Validation F1: 0.9157


Epoch 220, Train Loss: 0.5532, Validation Loss: 0.6924, Validation F1: 0.9100


Epoch 230, Train Loss: 0.5558, Validation Loss: 0.7062, Validation F1: 0.9071


Epoch 240, Train Loss: 0.5505, Validation Loss: 0.7051, Validation F1: 0.9090


Epoch 250, Train Loss: 0.5445, Validation Loss: 0.7373, Validation F1: 0.9106


Epoch 260 Saved best model. Best F1: 0.9162742641991372
Epoch 260, Train Loss: 0.5467, Validation Loss: 0.7275, Validation F1: 0.9163


Epoch 270, Train Loss: 0.5346, Validation Loss: 0.7228, Validation F1: 0.9061


Epoch 280, Train Loss: 0.5300, Validation Loss: 0.7257, Validation F1: 0.9127


Epoch 290, Train Loss: 0.5321, Validation Loss: 0.7518, Validation F1: 0.9100


Epoch 300, Train Loss: 0.5290, Validation Loss: 0.7701, Validation F1: 0.9126


Epoch 310, Train Loss: 0.5221, Validation Loss: 0.7617, Validation F1: 0.9132


Epoch 320, Train Loss: 0.5214, Validation Loss: 0.7213, Validation F1: 0.9125


Epoch 330, Train Loss: 0.5170, Validation Loss: 0.7519, Validation F1: 0.9129


Epoch 340, Train Loss: 0.5133, Validation Loss: 0.7564, Validation F1: 0.9031


Epoch 342 Saved best model. Best F1: 0.9164307364460663


Epoch 350, Train Loss: 0.5132, Validation Loss: 0.7521, Validation F1: 0.9158


Epoch 353 Saved best model. Best F1: 0.9166735740114207


Epoch 360, Train Loss: 0.5081, Validation Loss: 0.7845, Validation F1: 0.9132


Epoch 362 Saved best model. Best F1: 0.916858988176324


Epoch 370, Train Loss: 0.5074, Validation Loss: 0.7684, Validation F1: 0.9128


Epoch 379 Saved best model. Best F1: 0.9171573924243329


Epoch 380, Train Loss: 0.5031, Validation Loss: 0.7777, Validation F1: 0.9058


Epoch 390, Train Loss: 0.5048, Validation Loss: 0.7779, Validation F1: 0.9134


Epoch 393 Saved best model. Best F1: 0.91735009665473


Epoch 400, Train Loss: 0.4999, Validation Loss: 0.8110, Validation F1: 0.9135


Epoch 410, Train Loss: 0.4995, Validation Loss: 0.7730, Validation F1: 0.9106


Epoch 416 Saved best model. Best F1: 0.9174638272084433


Epoch 420, Train Loss: 0.4924, Validation Loss: 0.7886, Validation F1: 0.9134


Epoch 430, Train Loss: 0.4925, Validation Loss: 0.7834, Validation F1: 0.9118


Epoch 440, Train Loss: 0.4891, Validation Loss: 0.7870, Validation F1: 0.9141


Epoch 450, Train Loss: 0.4898, Validation Loss: 0.8166, Validation F1: 0.9153


Epoch 456 Saved best model. Best F1: 0.9176135949914116


Epoch 460, Train Loss: 0.4912, Validation Loss: 0.8148, Validation F1: 0.9087


Epoch 470, Train Loss: 0.4844, Validation Loss: 0.7886, Validation F1: 0.9159


Epoch 480, Train Loss: 0.4841, Validation Loss: 0.7935, Validation F1: 0.9156


Epoch 490, Train Loss: 0.4795, Validation Loss: 0.8081, Validation F1: 0.9145


Epoch 496 Saved best model. Best F1: 0.918431537480054


Epoch 500, Train Loss: 0.4787, Validation Loss: 0.8404, Validation F1: 0.9171


Epoch 510, Train Loss: 0.4775, Validation Loss: 0.8130, Validation F1: 0.9129


Epoch 520, Train Loss: 0.4773, Validation Loss: 0.7752, Validation F1: 0.9151


Epoch 530, Train Loss: 0.4756, Validation Loss: 0.8327, Validation F1: 0.9147


Epoch 540, Train Loss: 0.4740, Validation Loss: 0.8499, Validation F1: 0.9149


Epoch 550, Train Loss: 0.4722, Validation Loss: 0.8484, Validation F1: 0.9145


Epoch 560, Train Loss: 0.4704, Validation Loss: 0.8622, Validation F1: 0.9171


Epoch 570, Train Loss: 0.4707, Validation Loss: 0.8397, Validation F1: 0.9152


Epoch 580, Train Loss: 0.4696, Validation Loss: 0.8610, Validation F1: 0.9182


Epoch 590, Train Loss: 0.4673, Validation Loss: 0.8576, Validation F1: 0.9146


Epoch 600, Train Loss: 0.4698, Validation Loss: 0.9047, Validation F1: 0.9146


Epoch 610, Train Loss: 0.4662, Validation Loss: 0.8882, Validation F1: 0.9149


Epoch 620, Train Loss: 0.4673, Validation Loss: 0.8983, Validation F1: 0.9122


Epoch 630, Train Loss: 0.4657, Validation Loss: 0.8964, Validation F1: 0.9142


Epoch 640, Train Loss: 0.4650, Validation Loss: 0.8533, Validation F1: 0.9113


Epoch 650, Train Loss: 0.4642, Validation Loss: 0.8560, Validation F1: 0.9152


Epoch 660, Train Loss: 0.4636, Validation Loss: 0.8787, Validation F1: 0.9132


Epoch 670, Train Loss: 0.4624, Validation Loss: 0.9382, Validation F1: 0.9143


Epoch 680, Train Loss: 0.4606, Validation Loss: 0.8987, Validation F1: 0.9139


Epoch 690, Train Loss: 0.4636, Validation Loss: 0.8927, Validation F1: 0.9128


Epoch 700, Train Loss: 0.4603, Validation Loss: 0.8982, Validation F1: 0.9131


Epoch 710, Train Loss: 0.4589, Validation Loss: 0.8919, Validation F1: 0.9100


Epoch 720, Train Loss: 0.4601, Validation Loss: 0.9051, Validation F1: 0.9133


Epoch 730, Train Loss: 0.4575, Validation Loss: 0.9327, Validation F1: 0.9147


Epoch 740, Train Loss: 0.4573, Validation Loss: 0.9231, Validation F1: 0.9119


Epoch 750, Train Loss: 0.4587, Validation Loss: 0.9292, Validation F1: 0.9140


Epoch 760, Train Loss: 0.4576, Validation Loss: 0.9029, Validation F1: 0.9133


Epoch 770, Train Loss: 0.4565, Validation Loss: 0.9453, Validation F1: 0.9141


Epoch 780, Train Loss: 0.4561, Validation Loss: 0.9433, Validation F1: 0.9157


Epoch 790, Train Loss: 0.4542, Validation Loss: 0.9060, Validation F1: 0.9120


Epoch 800, Train Loss: 0.4537, Validation Loss: 0.9601, Validation F1: 0.9136


Epoch 810, Train Loss: 0.4560, Validation Loss: 0.8992, Validation F1: 0.9121


Epoch 820, Train Loss: 0.4550, Validation Loss: 0.9522, Validation F1: 0.9114


Epoch 830, Train Loss: 0.4533, Validation Loss: 0.9222, Validation F1: 0.9116


Epoch 840, Train Loss: 0.4530, Validation Loss: 0.9474, Validation F1: 0.9127


Epoch 850, Train Loss: 0.4551, Validation Loss: 0.9059, Validation F1: 0.9155


Epoch 860, Train Loss: 0.4549, Validation Loss: 0.9534, Validation F1: 0.9135


Epoch 870, Train Loss: 0.4555, Validation Loss: 0.9583, Validation F1: 0.9138


Epoch 880, Train Loss: 0.4544, Validation Loss: 0.9289, Validation F1: 0.9157


Epoch 890, Train Loss: 0.4545, Validation Loss: 0.9539, Validation F1: 0.9145


Epoch 900, Train Loss: 0.4535, Validation Loss: 0.9298, Validation F1: 0.9154


Epoch 910, Train Loss: 0.4539, Validation Loss: 0.9358, Validation F1: 0.9118


Epoch 920, Train Loss: 0.4520, Validation Loss: 0.9290, Validation F1: 0.9104


Epoch 930, Train Loss: 0.4538, Validation Loss: 0.9720, Validation F1: 0.9106


Epoch 940, Train Loss: 0.4525, Validation Loss: 0.9269, Validation F1: 0.9134


Epoch 950, Train Loss: 0.4552, Validation Loss: 0.9504, Validation F1: 0.9133


Epoch 960, Train Loss: 0.4537, Validation Loss: 0.9416, Validation F1: 0.9134


Epoch 970, Train Loss: 0.4545, Validation Loss: 0.9212, Validation F1: 0.9123


Epoch 980, Train Loss: 0.4529, Validation Loss: 0.9337, Validation F1: 0.9130


Epoch 990, Train Loss: 0.4537, Validation Loss: 0.9317, Validation F1: 0.9133


Model training completed and saved.


In [25]:
def plot_training_process(train_losses, val_losses, val_f1, saved_model_epochs):
    fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # Plot Train Loss
    axs[0].plot(train_losses, label='Train Loss', color='blue')
    axs[0].plot(val_losses, label='Validation Loss', color='red')
    axs[0].set_ylabel('Train Loss')
    axs[0].set_title('Training Loss')
    axs[0].legend()
    axs[0].grid()

    val_f1_weighted_history = []
    val_f1_micro_history = []
    val_f1_macro_history = []

    for val_f1_weighted, val_f1_micro, val_f1_macro in val_f1:
        val_f1_weighted_history.append(val_f1_weighted)
        val_f1_micro_history.append(val_f1_micro)
        val_f1_macro_history.append(val_f1_macro)
    
    # Plot Validation F1

    axs[1].plot(val_f1_weighted_history, label='Validation F1 Weighted', color='green')
    axs[1].plot(val_f1_micro_history, label='Validation F1 Micro', color='blue')
    axs[1].plot(val_f1_macro_history, label='Validation F1 Macro', color='red')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Validation F1')
    axs[1].set_title('Validation F1 Score')
    axs[1].legend()
    axs[1].grid()

    # Add scatter for saved model epochs (e.g., F1 weighted here)
    axs[1].scatter(saved_model_epochs, [val_f1_weighted_history[i] for i in saved_model_epochs],
                   color='black', marker='o', label='Saved Model')
    axs[1].legend()

    print(len(train_losses))

    plt.tight_layout()
    plt.show()

1000


In [26]:
plot_training_process(train_loss_history, val_loss_history, val_f1_history, saved_model_epochs)

In [27]:
from torch_geometric.utils import subgraph
from torch_geometric.data import Data
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

def eval(dataset, adversarial=False):

    best_model = EGraphSAGE(node_in_channels=num_features, 
                       edge_in_channels=num_features,
                       hidden_channels=best_hidden_dim, 
                       out_channels=num_classes).to(device)

    print("Loading model from", best_model_path)
    best_model.load_state_dict(th.load(best_model_path, weights_only=True))

    best_model.eval()

    print("inference start")
    with th.no_grad():
        all_pred_logits = []
        all_test_labels = []
        for G_pyg in tqdm(dataset, desc="Evaluation", leave=False):
            try:
                # Move the graph data to the device
                G_pyg = G_pyg.to(device)
                G_pyg.edge_label = G_pyg.edge_label.to(device)
                G_pyg.edge_attr = G_pyg.edge_attr.to(device)
                out = best_model(G_pyg)
                
            except Exception as forward_error:
                print(f"Error during forward/backward pass at {forward_error}")

            all_pred_logits.append(out.cpu())
            all_test_labels.append(G_pyg.edge_label.cpu())

        all_pred_logits = th.cat(all_pred_logits).to(device)
        all_test_labels = th.cat(all_test_labels).to(device)
        test_accuracy = compute_accuracy(all_pred_logits, all_test_labels)
        print(f'Test Accuracy: {test_accuracy:.4f}')

        pred_labels = all_pred_logits.argmax(dim=1).cpu()
        all_test_labels = all_test_labels.cpu()
    
    if adversarial:

        # Create a boolean mask where the label is NOT equal to the adversarial class
        adversarial_mask = all_test_labels == ADVERSARIAL_CLASS_LABEL

        # Print the class that the adversarial samples are classified as
        cm_adversarial = confusion_matrix(all_test_labels[adversarial_mask], pred_labels[adversarial_mask], labels=range(len(class_map) + 1))
        print("Adversarial confusion matrix:", cm_adversarial)

        # Apply the mask to both labels and predictions
        all_test_labels = all_test_labels[~adversarial_mask]
        pred_labels = pred_labels[~adversarial_mask]

    print("class_map", class_map)
    # Generate a report
    cm = confusion_matrix(all_test_labels, pred_labels, labels=range(len(class_map)))
    print(cm)


    report = classification_report(all_test_labels, pred_labels, target_names=class_map, digits=4)
    print(report)

print("Number of test graphs: ", len(test_graph_dataset))
eval(test_graph_dataset)


Number of test graphs:  278
Loading model from /vol/bitbucket/shc20/FYP/GNN-Adversarial-Attack/Models/E_GraphSAGE/logs/UNSW_NB15/window_raw_port/best_model_all_raw.pth
inference start


Test Accuracy: 0.9089
class_map ['Analysis' 'Backdoors' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
[[   85    68   114    39     8     4     0     2    10     1]
 [   50    69   111    25     7     1     0     3    19     2]
 [  186   130   841   270    66    34     1    22    25     7]
 [  236   279   978   757   168    94     8    98    75    18]
 [  111   162   120    58   345     8     0    32    44     5]
 [   24    50   125    82   116 15245     0    14    45     1]
 [   18     4    15    60   352     7 33003    56    55     1]
 [   18    24   156    60    36    10     1   166    14     5]
 [    0     0     0     3     2     0     0     6    24     0]
 [    0     1     0     0     3     0     0     0     0     2]]
                precision    recall  f1-score   support

      Analysis     0.1168    0.2568    0.1605       331
     Backdoors     0.0877    0.2404    0.1285       287
           DoS     0.3419    0.5316    0.4161      1582
   

In [28]:
def inject_traffic_to_attacker(graph, ratio=0.1, num_injected_nodes=1, is_attack=False):
    edge_index = graph.edge_index.clone()
    edge_attr = graph.edge_attr.clone()
    edge_label = graph.edge_label.clone()
    x = graph.x.clone()

    num_edges = edge_index.size(1)
    feature_dim = graph.x.size(1)

    # 1. Identify attacker nodes
    attacker_edges = (edge_label != BENIGN_CLASS_LABEL).nonzero(as_tuple=False).squeeze()
    attacker_nodes = th.unique(edge_index[:, attacker_edges])
    if attacker_nodes.numel() == 0:
        raise ValueError("No attacker nodes found.")

    # 2. Sample benign edge feature pool
    if is_attack:
        attack_edges = (edge_label != BENIGN_CLASS_LABEL).nonzero(as_tuple=False).squeeze()
        inject_edge_attr_pool = edge_attr[attack_edges]
    else:
        benign_edges = (edge_label == BENIGN_CLASS_LABEL).nonzero(as_tuple=False).squeeze()
        inject_edge_attr_pool = edge_attr[benign_edges]

    # 3. Inject new nodes
    original_num_nodes = x.size(0)

    new_node_feats = th.ones((num_injected_nodes, feature_dim))
    x = th.cat([x, new_node_feats], dim=0)

    # 4. Inject edges from injected nodes to attacker nodes
    num_to_inject = max(1, int(ratio * num_edges))
    new_edges = []
    new_attrs = []
    new_labels = []

    
    for _ in range(num_to_inject):
        src = random.randint(original_num_nodes, original_num_nodes + num_injected_nodes - 1)  # from injected nodes

        dst = attacker_nodes[random.randint(0, len(attacker_nodes) - 1)].item()

        new_edges.append([src, dst])
        attr = inject_edge_attr_pool[random.randint(0, len(inject_edge_attr_pool) - 1)]
        new_attrs.append(attr)
        new_labels.append(ADVERSARIAL_CLASS_LABEL)

    # Create a new empty graph to store the injected edges
    new_graph = Data()

    # 5. Merge into graph
    if new_edges:
        new_edges = th.tensor(new_edges, dtype=th.long).t().contiguous()
        new_attrs = th.stack(new_attrs)
        new_labels = th.tensor(new_labels, dtype=th.long)

        new_graph.edge_index = th.cat([edge_index, new_edges], dim=1)
        new_graph.edge_attr = th.cat([edge_attr, new_attrs], dim=0)
        new_graph.edge_label = th.cat([edge_label, new_labels], dim=0)
        new_graph.x = x

        # new_graph.first_injected_node_idx = original_num_nodes # Store injected node indices

    return new_graph


In [29]:
# Inject Attack Traffic to Attacker Nodes
injected_graph_dataset = [inject_traffic_to_attacker(g.cpu(), 0.1, num_injected_nodes=1, is_attack=True) for g in test_graph_dataset]
eval(injected_graph_dataset, adversarial=True)

Loading model from /vol/bitbucket/shc20/FYP/GNN-Adversarial-Attack/Models/E_GraphSAGE/logs/UNSW_NB15/window_raw_port/best_model_all_raw.pth
inference start


Evaluation:  38%|███▊      | 105/278 [00:00<00:00, 1041.40it/s]

Test Accuracy: 0.8203
Adversarial confusion matrix: [[   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [ 228  135  207  431 1872  709  900  422  552  104    0]]
class_map ['Analysis' 'Backdoors' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
[[   14    77    18   187     1     0    22     2    10     0]
 [    4    82    13   149     3     2    15     2    17     0]
 [   59   116   125  1108    24    25    93    14    18     0]
 

In [30]:
# Inject BENIGN Traffic to Attacker Nodes
injected_graph_dataset = [inject_traffic_to_attacker(g.cpu(), 0.1, num_injected_nodes=1, is_attack=False) for g in test_graph_dataset]
eval(injected_graph_dataset, adversarial=True)

Loading model from /vol/bitbucket/shc20/FYP/GNN-Adversarial-Attack/Models/E_GraphSAGE/logs/UNSW_NB15/window_raw_port/best_model_all_raw.pth
inference start


Test Accuracy: 0.8123
Adversarial confusion matrix: [[   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [  28   40   20  549  431   16 3760  224  406   86    0]]
class_map ['Analysis' 'Backdoors' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
[[   65    56    54    59    22     5    15    18    37     0]
 [   42    62    51    41    20     3    15    14    39     0]
 [   90   120   400   373    63    50   137   139   210     0]
 

In [31]:
def inject_random_nodes(graph, ratio=0.1, num_injected_nodes=1):
	edge_index = graph.edge_index.clone()
	edge_attr = graph.edge_attr.clone()
	edge_label = graph.edge_label.clone()
	x = graph.x.clone()

	num_edges = edge_index.size(1)
	feature_dim = graph.x.size(1)

	# 1. Inject new nodes
	original_num_nodes = x.size(0)
	new_node_feats = th.ones((num_injected_nodes, feature_dim))
	x = th.cat([x, new_node_feats], dim=0)

	# 2. Inject random edges
	num_to_inject = max(1, int(ratio * num_edges))
	new_edges = []
	new_attrs = []
	new_labels = []

	for _ in range(num_to_inject):
		src = random.randint(original_num_nodes, original_num_nodes + num_injected_nodes - 1)  # from injected nodes
		dst = random.randint(0, original_num_nodes - 1)  # to existing nodes

		new_edges.append([src, dst])
		attr = edge_attr[random.randint(0, len(edge_attr) - 1)]  # Randomly sample edge attributes
		new_attrs.append(attr)
		new_labels.append(ADVERSARIAL_CLASS_LABEL)  # Assign benign class label to new edges

	# 3. Merge into graph
	if new_edges:
		new_edges = th.tensor(new_edges, dtype=th.long).t().contiguous()
		new_attrs = th.stack(new_attrs)
		new_labels = th.tensor(new_labels, dtype=th.long)

		edge_index = th.cat([edge_index, new_edges], dim=1)
		edge_attr = th.cat([edge_attr, new_attrs], dim=0)
		edge_label = th.cat([edge_label, new_labels], dim=0)

	# Create a new graph with the injected nodes and edges
	new_graph = Data(
		edge_index=edge_index,
		edge_attr=edge_attr,
		edge_label=edge_label,
		x=x
	)

	return new_graph

In [32]:
# Inject Random Nodes in the graph
injected_graph_dataset = [inject_random_nodes(g.cpu(), 0.1, num_injected_nodes=1) for g in test_graph_dataset]
eval(injected_graph_dataset, adversarial=True)

Loading model from /vol/bitbucket/shc20/FYP/GNN-Adversarial-Attack/Models/E_GraphSAGE/logs/UNSW_NB15/window_raw_port/best_model_all_raw.pth
inference start


Evaluation:   0%|          | 0/278 [00:00<?, ?it/s]

Test Accuracy: 0.8189
Adversarial confusion matrix: [[   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [  40  102  171  508  471  358 3539  203  147   21    0]]
class_map ['Analysis' 'Backdoors' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal'
 'Reconnaissance' 'Shellcode' 'Worms']
[[   43    72    73    85     2     0    31    10    15     0]
 [   21    77    65    59     4     1    24    12    24     0]
 [  132    80   528   578    38    19    71    81    54     1]
 